# Messaging services

- store and pull messages
- manage read receipts

In [ ]:
//| export

import type { Database } from "tinychat/db.ts";
import { Record as Message } from "tinychat/api/types/chat/tinychat/core/message.ts";
import { ChannelView } from "tinychat/api/types/chat/tinychat/server/defs.ts";

const get_time_us = (): string => `${new Date().getTime() * 1000}`;

export class Messaging {
  constructor(protected db: Database) {}

  public markAllMessagesAsRead({ channel, user } : { channel: string, user: string }) {
    this.db
      .prepare(`INSERT OR REPLACE INTO read_receipts (channel, user, time_us) VALUES (:channel, :user, :time)`)
      .run({ channel, user, time: get_time_us() });
  }

  public getChannels({ server, viewer } : { server: string, viewer?: string | undefined}) : ChannelView[] {
     let base = "SELECT * FROM channel_view WHERE server = :server";
     if (viewer) {
       base += " AND user = :viewer";
     }
     return this.db
       .prepare(base)
       .all<{
         uri: string;
         name: string;
         latest_message_received_time_us: string | null;
         last_message_read_time: string | null;
       }>(Object.assign({ server }, viewer ? { viewer } : {}))
       .map((rec) => ({
        uri: rec.uri,
        name: rec.name,
        lastMessageReadTime: rec.last_message_read_time || undefined,
        latestMessageReceivedTimeUs: rec.latest_message_received_time_us || undefined,
       }));
  }

  public receiveMessage(
    { m, uri, sender, time_us }: {
      m: Message;
      uri: string;
      sender: string;
      time_us: string;
    },
  ) {
    this.db.prepare(`
      INSERT INTO messages (uri, channel, server, text, sender, created_at, time_us) VALUES (
        :uri, :channel, :server, :text, :sender, :created_at, :time_us
      )`).run({
      uri,
      channel: m.channel,
      server: m.server,
      text: m.text,
      sender,
      created_at: m.createdAt,
      time_us: time_us,
    });
  }
}

In [ ]:
//| export

import { getDatabase} from "tinychat/db.ts";

class TestMessaging extends Messaging {
  constructor() {
    super(getDatabase({ reset: true }));
  }

  public static server: string = "at://server-1";
  public static user1: string = "did:1";
  public static user2: string = "did:2";
  public static channel1: string = "at://channel-1";
  public static channel2: string = "at://channel-2";

  public user1MessagesChannel1(text: string) {
    this.receiveMessage({
      m: {
        channel: "at://channel-1",
        server: "at://server-1",
        text,
        createdAt: new Date().toISOString(),
      },
      uri: "at://message-1",
      sender: TestMessaging.user1,
      time_us: get_time_us(),
    });
  }

  public static setup(): TestMessaging {
    const service = new TestMessaging();
    // insert 2 test users
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO users (did, handle, display_name, avatar, description) VALUES (
          :did, :handle, :display_name, :avatar, :description
        )
      `
        )
        .run({
          did: `did:${i}`,
          handle: `user-${i}`,
          display_name: `User ${i}`,
          avatar: `http://google.com/avatar-${i}.jpeg`,
          description: `description ${i}`,
        });
    });
    // create test server
    service.db
      .prepare(
        `
      INSERT INTO servers (uri, name, creator) VALUES (
        :uri, :name, :creator
      )
    `
      )
      .run({
        uri: "at://server-1",
        name: "Test Server",
        creator: "did:1",
      });

    // create memberships for both users
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO server_memberships (user, server) VALUES (
          :user, :server
        )
      `
        )
        .run({
          user: `did:${i}`,
          server: "at://server-1",
        });
    });

    // setup channels
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO channels (uri, name, server) VALUES (
          :uri, :name, :server
        )
      `
        )
        .run({
          uri: `at://channel-${i}`,
          name: `channel ${i}`,
          server: "at://server-1",
        });
    });
    return service;
  }
}


In [ ]:
// import { j } from "@jurassic/jurassic";
// j.initialize("messaging.ipynb");
// await j`hey`;


Hi! I'm ready to help you with the messaging module. You can ask me about the code, request changes, or help with documentation. What would you like to work on?

In [ ]:
import { assert } from "asserts";

Deno.test("test channel tracks last message received", () => {
  const messaging = TestMessaging.setup();
  messaging.user1MessagesChannel1("hello world");

  [undefined, TestMessaging.user2].forEach((viewer) => {
    const channels = messaging.getChannels({
      server: TestMessaging.server,
      viewer,
    });

    assert(
      channels.find((c) => c.uri === TestMessaging.channel1)
        ?.latestMessageReceivedTimeUs,
      "channel 1 has last message received time set"
    );

    assert(
      !channels.find((c) => c.uri === TestMessaging.channel2)
        ?.latestMessageReceivedTimeUs,
      "channel 2 does NOT have last message received time set"
    );
  })
});

Deno.test("test mark all messages as read", () => {
  const messaging = TestMessaging.setup();
  messaging.user1MessagesChannel1("hello world");

  const channel1ForUser2 = messaging
    .getChannels({
      server: TestMessaging.server,
      viewer: TestMessaging.user2,
    })
    .find((c) => c.uri === TestMessaging.channel1);
  
    assert(channel1ForUser2, "got channel 1 for user 2 info");
    assert(!channel1ForUser2.lastMessageReadTime, "last message read time is not set");

  messaging.markAllMessagesAsRead({
    channel: TestMessaging.channel1,
    user: TestMessaging.user2,
  });

    assert(messaging
    .getChannels({
      server: TestMessaging.server,
      viewer: TestMessaging.user2,
    })
    .find((c) => c.uri === TestMessaging.channel1)!.lastMessageReadTime, "last message read time is set");

    // make sure we can mark all messages as read multiple times
    messaging.markAllMessagesAsRead({
      channel: TestMessaging.channel1,
      user: TestMessaging.user2,
    });
});